In [1]:
%load_ext autoreload
%autoreload 2

In [104]:
from wdtools import *

In [22]:
setID = 'Set007'; nm_to_add = 4972

In [69]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add, raw=False)

In [5]:
file = outpath + f'\\to_review\\re_mapping_{setID}.txt'
with open(file) as f:
    remapIDs = f.readlines()

In [6]:
wd.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [78]:
len(wd.wetdet_delin_number.unique())

299

In [14]:
partialIDs = list(wd[wd.notes != ''].wetdet_delin_number.unique())

In [97]:
unmatched = pd.read_csv(outpath+f'\\to_review\\unmatched_df_{setID}_2.csv')

In [98]:
unmatched.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'DocumentName', 'DecisionLink', 'is_batch_file', 'status_name',
       'received_date', 'Coord-Source', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [100]:
unmatchedIDs = list(unmatched.wetdet_delin_number.unique())

In [101]:
qaqcIDs = remapIDs[0].split(', ') + partialIDs + unmatchedIDs

In [102]:
check_cnt = wd[wd.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['wetdet_delin_number']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'wdID', 'county':'count'})

In [103]:
check_cnt[check_cnt['count']>1]

,wdID,count


In [105]:
test = update_QAQC_data(setID=setID, wd=wd, 
                        totcol='P2_2008_16', 
                        qaqc_col='P2_ToMap', 
                        export=False)

ignore below codes

In [106]:
qaqc_df = wd[wd.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'QAQC_count'})

In [107]:
qaqc_df

,county,QAQC_count
0,Clatsop,91
1,Deschutes,25


In [37]:
total_df = wd[['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'total_count'})

In [38]:
total_df

,county,total_count
0,Clatsop,201
1,Deschutes,98


In [31]:
gdf = gpd.read_file(inpath+'\\reporting\\WD_Counts.shp')

In [34]:
gdf.columns

Index(['County', 'Note', 'P1_2017_22', 'P1_ToMap', 'P2_2008_16', 'P2_ToMap',
       'P3_1990_07', 'P3_ToMap', 'Tot_ToMap', 'Tot_Count', 'QAQC_Ratio',
       'Auto_Comp', 'geometry'],
      dtype='object')

In [58]:
sel = qaqc_df.county.unique()

In [52]:
total_df.loc[total_df.county.isin(sel), 
             ['county', 'total_count']]

,county,total_count
0,Clatsop,201
1,Deschutes,98


In [54]:
gdf.loc[gdf.County.isin(sel), 
        ['County', 'P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
9,Deschutes,132,66
35,Clatsop,275,137


In [44]:
wdcnt_dict = total_df.set_index('county').to_dict(orient='dict')['total_count']

In [108]:
qccnt_dict = qaqc_df.set_index('county').to_dict(orient='dict')['QAQC_count']

In [46]:
qccnt_dict

{'Clatsop': 87, 'Deschutes': 23}

In [55]:
gdf.loc[gdf.County.isin(sel), 'P2_2008_16'] = gdf[gdf.County.isin(sel)].County.map(wdcnt_dict)

In [109]:
gdf.loc[gdf.County.isin(sel), 'P2_ToMap'] = gdf[gdf.County.isin(sel)].County.map(qccnt_dict)

In [110]:
gdf.loc[gdf.County.isin(sel), 
        ['County', 'P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
9,Deschutes,98,25
35,Clatsop,201,91


In [59]:
gdf.loc[:, 'Tot_ToMap'] =  gdf.P1_ToMap + gdf.P2_ToMap + gdf.P3_ToMap
gdf.loc[:, 'Tot_Count'] =  gdf.P1_2017_22 + gdf.P2_2008_16 + gdf.P3_1990_07
gdf.loc[:, 'QAQC_Ratio'] = gdf.Tot_ToMap / gdf.Tot_Count
gdf.loc[:, 'Auto_Comp'] = gdf.Tot_Count - gdf.Tot_ToMap

In [112]:
gdf.to_file(inpath+'\\reporting\\WD_Counts.shp')

In [111]:
test.loc[test.County.isin(sel), 
        ['County', 'P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
9,Deschutes,98,25
35,Clatsop,201,91


In [113]:
os.remove(inpath+"\\reporting\\WD_Counts.zip")

In [114]:
file_name_list = glob.glob(inpath+"\\reporting\\WD_Counts.*")
zip_file_name = inpath+"\\reporting\\WD_Counts.zip"
file_compress(file_name_list, zip_file_name)

 *** Input File name passed for zipping - ['L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.cpg', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.dbf', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.prj', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shp', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shx']
 *** out_zip_file is - L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\reporting\WD_Counts.zip
 *** Processing file L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\reporting\WD_Counts.cpg
 *** Processing file L:\NaturalResources\Wetlands\Loc